# Llama3 大模型微调实战

Llama3 是 Meta 于2024年4月开放的 Llama 系列的最新模型。基于超过 15T token 训练，相当于 Llama 2 数据集的 7 倍多；支持 8K 长文本，改进的 tokenizer 具有 128K token 的词汇量，可实现更好的性能。Llama3 提供两个版本: 8B版本适合在消费级GPU上高效部署和开发，70B版本则专为大规模AI应用设计。每个版本都包括基础和指令调优两种形式：
* Meta-Llama-3-8b: 8B 基础模型
* Meta-Llama-3-8b-instruct: 8B 基础模型的指令调优版
* Meta-Llama-3-70b: 70B 基础模型
* Meta-Llama-3-70b-instruct: 70B 基础模型的指令调优版

本文将以Llama-3-8B-Instruct为例，为您介绍如何在PAI-DSW中微调Llama3大模型。

## 运行环境要求

* Python环境3.9以上，推荐使用V100(16GB)或更高显存的GPU。

* 镜像推荐使用如下URL，其中REGION为DSW实例所属区域，例如cn-shanghai、cn-hangzhou等。

dsw-registry-vpc.REGION.cr.aliyuncs.com/pai-training-algorithm/llm_deepspeed_peft:v0.0.3

## 准备工作
### 下载模型

**注：使用此模型受Meta许可证的约束。在使用模型前，请务必阅读[Meta官方许可证](https://huggingface.co/meta-llama/Meta-Llama-3-70B/blob/main/LICENSE)。**

首先，需要下载模型，您可以[向Meta申请下载模型](https://llama.meta.com/llama-downloads)，或者根据下文代码通过ModelScope下载。

In [1]:
!pip install modelscope==1.12.0 transformers==4.37.0

Looking in indexes: https://mirrors.aliyun.com/pypi/simple
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.4/8.4 MB 2.5 MB/s eta 0:00:0000:0100:01
DEPRECATION: pytorch-lightning 1.7.7 has a non-standard dependency specifier torch>=1.9.*. pip 24.0 will enforce this behaviour change. A possible replacement is to upgrade to a newer version of pytorch-lightning or contact the author to suggest that they release a version with a conforming dependency specifiers. Discussion can be found at https://github.com/pypa/pip/issues/12063
  Attempting uninstall: transformers
    Found existing installation: transformers 4.37.2
    Uninstalling transformers-4.37.2:
      Successfully uninstalled transformers-4.37.2

[notice] A new release of pip is available: 23.3.2 -> 24.0
[notice] To update, run: pip install --upgrade pip


In [2]:
from modelscope.hub.snapshot_download import snapshot_download
snapshot_download('LLM-Research/Meta-Llama-3-8B-Instruct', cache_dir='.', revision='master')
# snapshot_download('LLM-Research/Meta-Llama-3-8B', cache_dir='.', revision='master')
# snapshot_download('LLM-Research/Meta-Llama-3-70B-Instruct', cache_dir='.', revision='master')
# snapshot_download('LLM-Research/Meta-Llama-3-70B', cache_dir='.', revision='master')

2024-05-02 14:57:49,459 - modelscope - INFO - PyTorch version 2.1.2+cu121 Found.
2024-05-02 14:57:49,463 - modelscope - INFO - TensorFlow version 2.14.0 Found.
2024-05-02 14:57:49,464 - modelscope - INFO - Loading ast index from /mnt/workspace/.cache/modelscope/ast_indexer
2024-05-02 14:57:49,464 - modelscope - INFO - No valid ast index found from /mnt/workspace/.cache/modelscope/ast_indexer, generating ast index from prebuilt!
2024-05-02 14:57:50,061 - modelscope - INFO - Loading done! Current index file version is 1.12.0, with md5 509123dba36c5e70a95f6780df348471 and a total number of 964 components indexed
/opt/conda/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Downloading: 100%|██████████| 654/654 [00:00<00:00, 3.61MB/s]
Downloading: 100%|██████████| 48.0/48.0 [00:00<00:00, 320kB/s]
Downloading: 

'./LLM-Research/Meta-Llama-3-8B-Instruct'

### 准备数据集

接下来，准备微调示例数据集。本文准备了英文诗歌数据集，来微调 Llama3 大模型，使其提高其生成诗歌的表现能力。

您也可以参考该数据集的格式，根据自己的使用场景，准备所需的数据集。通过微调，能够提高大语言模型在特定任务上的回答准确性。

In [3]:
!wget https://atp-modelzoo-sh.oss-cn-shanghai.aliyuncs.com/tutorial/llm_instruct/en_poetry_train.json

--2024-05-02 15:00:08--  https://atp-modelzoo-sh.oss-cn-shanghai.aliyuncs.com/tutorial/llm_instruct/en_poetry_train.json
正在解析主机 atp-modelzoo-sh.oss-cn-shanghai.aliyuncs.com (atp-modelzoo-sh.oss-cn-shanghai.aliyuncs.com)... 47.101.88.27
正在连接 atp-modelzoo-sh.oss-cn-shanghai.aliyuncs.com (atp-modelzoo-sh.oss-cn-shanghai.aliyuncs.com)|47.101.88.27|:443... 已连接。
已发出 HTTP 请求，正在等待回应... 200 OK
长度： 636887 (622K) [application/json]
正在保存至: ‘en_poetry_train.json’

en_poetry_train.jso 100%[===================>] 621.96K  --.-KB/s    用时 0.02s   

2024-05-02 15:00:09 (24.8 MB/s) - 已保存 ‘en_poetry_train.json’ [636887/636887])



## 微调模型

接下来，基于已有的训练脚本`/ml/code/sft.py`，进行模型的LoRA轻量化训练。在训练结束之后，我们将模型参数进行量化，以便使用更少显存进行推理。

示例使用的参数解释如下，请您根据实际情况进行修改：

- `accelerate launch`命令行工具用于在多GPU中启动和管理深度学习训练脚本。
    - `num_processes` 1：设置并行处理的进程数量为 1，即不进行多进程并行处理。
    - `config_file` `/ml/code/multi_gpu.yaml`：指定配置文件的路径。
    - `/ml/code/sft.py`：指定要运行的 Python 脚本的路径。
    
    
脚本`/ml/code/sft.py`接受的参数：
- `--model_name` `./LLM-Research/Meta-Llama-3-8B-Instruct/`：指定预训练模型的路径。
- `--model_type` `llama`：指定模型的类型，这里是 llama。
- `--train_dataset_name` `chinese_medical_train_sampled.json`：指定训练数据集的路径。
- `--num_train_epochs` 3：设置训练的轮次为 3。
- `--batch_size` 8：设置批处理的大小为 8。
- `--seq_length` 128：设置序列的长度为 128。
- `--learning_rate` 5e-4：设置学习率为 0.0005。
- `--lr_scheduler_type` linear：设置学习率调度器类型为线性。
- `--target_modules` k_proj o_proj q_proj v_proj：指定在微调中需要特别关注的模型模块。
- `--output_dir` `lora_model/`：指定输出目录路径，微调后的模型将被保存在这里。
- `--apply_chat_template`：指定训练时应用聊天模板。
- `--use_peft`：在训练过程中使用参数有效调优PEFT（Parameter-Efficient Fine-Tuning）方法。
- `--load_in_4bit`：指示模型权重载入时使用 4 位精度，减少内存消耗。
- `--peft_lora_r` 32：如果使用了 LoRA（Low-Rank Adaptation）作为参数有效调优方法的一部分，这会指定 LoRA 的秩为 32。
- `--peft_lora_alpha` 32：设置 LoRA 参数的另一部分，alpha 的大小为 32。

当运行`accelerate launch`命令时，它会使用这些参数来启动指定的 Python 脚本，并且根据`multi_gpu.yaml`配置文件中的设置，在计算资源允许的范围内进行训练。

In [6]:
! accelerate launch --num_processes 1 --config_file ../ml-code/multi_gpu.yaml ../ml-code/sft.py \
    --model_name  ./LLM-Research/Meta-Llama-3-8B-Instruct/ \
    --model_type llama \
    --train_dataset_name en_poetry_train.json \
    --num_train_epochs 3 \
    --batch_size 8 \
    --seq_length 128 \
    --learning_rate 5e-4 \
    --lr_scheduler_type linear \
    --target_modules k_proj o_proj q_proj v_proj \
    --output_dir lora_model/ \
    --apply_chat_template \
    --use_peft \
    --load_in_4bit \
    --peft_lora_r 32 \
    --peft_lora_alpha 32

2024-05-02 16:25:43.704595: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2024-05-02 16:25:44.195998: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-05-02 16:25:44.196038: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-05-02 16:25:44.199348: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-05-02 16:25:44.497531: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2024-05-02 16:25:44.499983: I tensorflow/core/platform/cpu_feature_guard.cc:182] This Tens

接下来，将LoRA权重与基础模型融合。示例使用的参数解释如下：

* RANK=0：环境变量RANK用于分布式训练中，来表示当前进程在所有进程中的序号。设为0表明它是单进程或者是分布式训练中的主进程。
* python `/ml/code/convert.py`：执行convert.py脚本，用于权重转换或其他转换工作。
* `--model_name` `./LLM-Research/Meta-Llama-3-8B-Instruct/`：指定基础模型的路径。
* `--model_type` llama：指定模型类型，这里是llama。
* `--output_dir` `trained_model/`：指定转换后的模型和权重应该输出保存的目录。
* `--adapter_dir` `lora_model/`：指定包含LoRA适配器权重的目录。

In [7]:
! RANK=0 python ../ml-code/convert.py \
    --model_name ./LLM-Research/Meta-Llama-3-8B-Instruct/ \
    --model_type llama \
    --output_dir trained_model/ \
    --adapter_dir lora_model/

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Loading checkpoint shards: 100%|██████████████████| 4/4 [00:04<00:00,  1.20s/it]


## 推理模型 

微调好模型之后，我们使用模型进行推理，来验证微调的效果。这里我们让模型写一首关于春天的诗歌，生成的作品效果也很好。

In [ ]:
import torch, transformers

# model_id = "./LLM-Research/Meta-Llama-3-8B-Instruct/"
model_id = "./trained_model/"
pipeline = transformers.pipeline(
    "text-generation",
    model=model_id,
    model_kwargs={"torch_dtype": torch.bfloat16},
    device="cuda",
)

messages = [
    {"role": "user", "content": "Write a poem on a topic 'Spring' "},
]

prompt = pipeline.tokenizer.apply_chat_template(
        messages, 
        tokenize=False, 
        add_generation_prompt=True
)

terminators = [
    pipeline.tokenizer.eos_token_id,
    pipeline.tokenizer.convert_tokens_to_ids("<|eot_id|>")
]

outputs = pipeline(
    prompt,
    max_new_tokens=1024,
    eos_token_id=terminators,
    do_sample=True,
    temperature=0.6,
    top_p=0.9,
)
print(outputs[0]["generated_text"][len(prompt):])

Loading checkpoint shards:  50%|█████     | 2/4 [00:55<00:55, 27.71s/it]

## 部署模型

您可以将微调后的模型权重上传至OSS，参考[5分钟使用EAS一键部署LLM大语言模型应用](https://help.aliyun.com/zh/pai/use-cases/deploy-llm-in-eas?spm=a2c4g.11186623.0.0.43c15699O8N4hA)，使用EAS ChatLLM部署微调后的Llama3模型服务。